In [1]:
# LIBRARY_PATH = '/home/deleidos/Notebooks/Newspaper/SemanticEstimation'
LIBRARY_PATH = "/home/deleidos/Notebooks/lib"

import sys
import os

module_path = os.path.abspath(os.path.join(LIBRARY_PATH))
if module_path not in sys.path:
    sys.path.append(module_path)
print (f"sys.path : {sys.path}") 

sys.path : ['/home/deleidos/Notebooks/Newspaper/SemanticEstimation', '/opt/conda/lib/python38.zip', '/opt/conda/lib/python3.8', '/opt/conda/lib/python3.8/lib-dynload', '', '/opt/conda/lib/python3.8/site-packages', '/opt/conda/lib/python3.8/site-packages/IPython/extensions', '/home/deleidos/.ipython', '/home/deleidos/Notebooks/lib']


In [2]:
from APIutils import *
import newspaper
from newspaper import Article 
from keybert import KeyBERT

import json 
import boto3 
import logging 
import uuid 
import time
from datetime import date, datetime 
import datetime
from pathlib import Path 
import os 
import glob
import nltk
from bertopic import BERTopic
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

from transformers import pipeline

classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
import pandas as pd
import contractions

# nltk.download('punkt')
# setup widget import
from ipywidgets import widgets,Layout
from IPython.display import display

from utils import *
import random
from timeit import default_timer as timer
import time
import datetime


#LOCALS
RESOURCE = 'dynamodb'
TABLE = "Articles-tfoo6wfq2zbxfdwl3z47nsgtrq-dev"
REGION = "us-east-1"

CSVPath = "/home/deleidos/Notebooks/CSVData"
JSONPath = "/home/deleidos/Notebooks/JSONData"
MODELPATH= "/home/deleidos/Notebooks/Models"
TRANSFORMERMODEL = 'bert-base-nli-mean-tokens'
from dynamoDButils import *
from APIutils import *
from seedterms import *


def complete(state="complete"):
    print(f"\nCell {state} @ {(datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))}")
complete(state='imports done')

[nltk_data] Downloading package punkt to /home/deleidos/nltk_data...
[nltk_data]   Package punkt is already up-to-date!



Cell imports done @ 2023-02-14 16:13:36


### Define the 3 different scoring methods

Create dictionary to hold scores and topics to eventually be displayed in a df.

Old scoring method is the original method used to select articles for the newsfeed site. 
The restrictive scoring (or modified BoW) approach was developed during researching/evaluating methods for the Semantic Estimation paper.
The weighted scoring method provides a potential median to these aforementioned techniques.

In [30]:
# Create empty dict to add the scores to 
test_dict = {
    "summary":[], "oldScore":[], "modBoW":[], "weightScore":[], "topic1":[], "topic2":[]
}

In [31]:
# Original method for scoring 
def getOldScore(classify):
    THRESHOLD = 0.8
    
    score = round (
        (   round (classify['scores'][classify['labels'].index('technology')], 2 )
               + round (classify['scores'][classify['labels'].index('innovation')], 2 ) 
               + round (classify['scores'][classify['labels'].index('politics')], 2 )
               + round (classify['scores'][classify['labels'].index('electronics')], 2 )
               + round (classify['scores'][classify['labels'].index('federal')], 2 )
               + round (classify['scores'][classify['labels'].index('public')], 2 )
               + round (classify['scores'][classify['labels'].index('government')], 2 )
               + round (classify['scores'][classify['labels'].index('data')], 2 )
               + round (classify['scores'][classify['labels'].index('ai')], 2 )
               + round (classify['scores'][classify['labels'].index('application')], 2 )
               + round (classify['scores'][classify['labels'].index('security')], 2 )
               + round (classify['scores'][classify['labels'].index('cyber security')], 2 )
               + round (classify['scores'][classify['labels'].index('business innovation')], 2 )
            )
        ,2)
    
    return score
    

In [32]:
# Method stemming from modified BoW approach
def restrictiveScore(df):
    THRESHOLD = 0.4

    cumulative = 0.0
    topic1 = df.iloc[0] ['label']
    topic2 = df.iloc[1] ['label']
    score1 = round(df.iloc[0] ['score'],2)
    score2 = round(df.iloc[1] ['score'],2)
    cumulative = score1 + score2
    
    return (topic1, topic2, cumulative)

def format_results(classify):
    labels = classify['labels']
    scores = classify['scores']
    scores_csv = []
    for l in enumerate(classify['labels']):
        # note labels are in the form (0, 'public'), so can just use the oth term in l as index to scores
        scores_csv.append(
            {
                "label" : l[1],
                "score" : scores[l[0]]  
            }
        )
    return pd.DataFrame(scores_csv)

In [33]:
def weightedScore(classify):
    # Weighted topic ranking scoring system 
    CUTOFF = 0.45

    # multiplier for each topic score is indicative of the level of importance of the category
    weight_score = round ((
        (classify['scores'][classify['labels'].index('technology')] * 1.0) +
        (classify['scores'][classify['labels'].index('innovation')] * 1.0) +
        (classify['scores'][classify['labels'].index('ai')] * 0.8) +
        (classify['scores'][classify['labels'].index('business innovation')] * 0.8) +
        (classify['scores'][classify['labels'].index('science')] * 0.8) +
        (classify['scores'][classify['labels'].index('federal')] * 0.6) +
        (classify['scores'][classify['labels'].index('data')] * 0.6) +
        (classify['scores'][classify['labels'].index('cyber security')] * 0.6) +
        (classify['scores'][classify['labels'].index('government')] * 0.6) +
        (classify['scores'][classify['labels'].index('security')] * 0.5) +
        (classify['scores'][classify['labels'].index('electronics')] * 0.5) +
        (classify['scores'][classify['labels'].index('networks')] * 0.5) +
        (classify['scores'][classify['labels'].index('public')] * 0.4) +

        # labels indicative of off-topic articles given a negative multiplier
        (classify['scores'][classify['labels'].index('politics')] * -0.4) +
        (classify['scores'][classify['labels'].index('social')] * -0.6) +
        (classify['scores'][classify['labels'].index('games')] * -0.8) +
        (classify['scores'][classify['labels'].index('entertainment')] * -0.8) +
        (classify['scores'][classify['labels'].index('sports')] * -0.8) +
        (classify['scores'][classify['labels'].index('advertisement')] * -1.0) 
        ), 2)

    return (weight_score)


### Run candidate articles through the methods

Load candidate article summaries and run through each score method to compare results.

In [4]:
jsonfiles = [f for f in listdir(JSONPath) if isfile(join(JSONPath, f))]
jsonfiles.sort()
jsonfiles = [x for x in jsonfiles if (os.path.splitext(x)[1][1:]).lower() == "json"]
jsnWD = widgets.Select(
    options=jsonfiles,
    value=jsonfiles[0],
    description='Choose ID:',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
        layout=Layout(width='50%', height='256px')
)
def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
#         print ("changed to %s" % change['new'])
        selection = change['new']

jsnWD.observe(on_change)
display(jsnWD)

Select(description='Choose ID:', layout=Layout(height='256px', width='50%'), options=('2000CWE.json', 'NOTsema…

In [18]:
jsnf = jsnWD.value
print (f"{jsnf=} ")

jsnfile = osp.join (JSONPath,jsnf)
xSummaries = pd.read_json(jsnfile)
xSummaries.head()

jsnf='semanticSimArticlesTestingDF.json' 


,article,summary,hf1,hf2,hf3,hf4,hf5,hfAvg,kb1,kb2,kb3,kb4,kb5,kbAvg,topic1,score1,topic2,score2
0,https://datainnovation.org/2022/11/visualizing...,economist created visualization showing politi...,0.5627,0.5556,0.5153,0.5132,0.4939,0.528140,0.408926,0.408615,0.395756,0.385963,0.365358,0.392923,economics,0.629220,politics,0.167246
1,https://venturebeat.com/ai/stability-ai-looks-...,among organization building generative technol...,0.9986,0.6797,0.6685,0.6572,0.6319,0.659325,0.515490,0.456996,0.455971,0.445295,0.442166,0.450107,technology,0.585379,innovation,0.180912
2,https://venturebeat.com/ai/openai-ceo-admits-c...,watch demand session today watch herealtman la...,0.5767,0.5576,0.5529,0.5504,0.5455,0.551600,0.549421,0.546504,0.544978,0.534699,0.533749,0.541870,entertainment,0.176518,social,0.170545
3,https://venturebeat.com/ai/generative-ai-may-b...,generative going completely alter marketing la...,0.7521,0.7365,0.7272,0.7240,0.7216,0.732280,0.519292,0.496403,0.488997,0.482039,0.477190,0.486157,innovation,0.602014,business innovation,0.258873
4,https://www.businessinsider.com/moderna-ceo-in...,personalized cancer vaccine could provide new ...,0.5688,0.5227,0.4532,0.4527,0.4452,0.488520,0.428122,0.427320,0.422924,0.419798,0.395541,0.424541,innovation,0.430343,technology,0.140970


In [19]:
xSentences = preProcessing(xSummaries)


Cell models @ 2023-02-14 18:49:23


In [22]:
xSentences[0]

'economist created visualization showing politician twitter follower changed elon musk acquired platform visualization show percent change individual account party average october october according visualization republican gained around follower musk acquired twitter democrat lost follower take look'

First run each summary through the classifier, then score the article with each method outlined above. 

Add the results to a dictionary for eventual display in a DF.

In [21]:
def loadCandidateArticles(summaries):
    # Create a zero shot classifier with specific category labels
    classify = []
    candidate_labels = [ 'technology', 'innovation','politics', 'electronics'
                        ,'entertainment', 'economics', 'advertisement', 'social'
                        ,'games', 'sports', 'health', 'science', 'business innovation'
                        ,'federal', 'public', 'government', 'data', 'ai'
                        , 'application','security', 'cyber security', 'networks' ]
    
    for summary in summaries:
        article = classifier( summary
                                      , candidate_labels
                                      ,device=0     
                             )
        classify.append(article)
   
    return classify

In [24]:
%%time
classify = loadCandidateArticles(xSentences)

CPU times: user 1h 1min 38s, sys: 4.61 s, total: 1h 1min 43s
Wall time: 30min 52s


In [29]:
classify[0:5]

[{'sequence': 'economist created visualization showing politician twitter follower changed elon musk acquired platform visualization show percent change individual account party average october october according visualization republican gained around follower musk acquired twitter democrat lost follower take look',
  'labels': ['economics',
   'data',
   'politics',
   'public',
   'science',
   'innovation',
   'ai',
   'application',
   'social',
   'technology',
   'networks',
   'business innovation',
   'government',
   'entertainment',
   'security',
   'advertisement',
   'games',
   'electronics',
   'health',
   'federal',
   'cyber security',
   'sports'],
  'scores': [0.4189926087856293,
   0.19949834048748016,
   0.11136756092309952,
   0.059279732406139374,
   0.047800224274396896,
   0.0310827624052763,
   0.026711858808994293,
   0.025122009217739105,
   0.019509458914399147,
   0.015515671111643314,
   0.009398100897669792,
   0.009149364195764065,
   0.0074247582815587

In [34]:
for indx in range(len(classify)): 
    score = getOldScore(classify[indx])
    
    df = pd.DataFrame()
    df = format_results(classify[indx])
    rtn = restrictiveScore(df)
    
    weight_score = weightedScore(classify[indx])
    
    # Add the re-evalaution scores and summary to dictionary
    test_dict['summary'].append(classify[indx]['sequence'])
    test_dict['oldScore'].append(score)
    test_dict['modBoW'].append(rtn[2])
    test_dict['weightScore'].append(weight_score)
    test_dict['topic1'].append(rtn[0])
    test_dict['topic2'].append(rtn[1])
       

In [42]:
# convert dictionary to pandas df
scoresdf = pd.DataFrame(test_dict)
scoresdf.head()

,summary,oldScore,modBoW,weightScore,topic1,topic2
0,economist created visualization showing politi...,0.50,0.62,0.21,economics,data
1,among organization building generative technol...,0.85,0.58,0.66,technology,innovation
2,watch demand session today watch herealtman la...,0.72,0.44,0.29,public,ai
3,generative going completely alter marketing la...,0.92,0.57,0.51,innovation,application
4,personalized cancer vaccine could provide new ...,0.71,0.44,0.57,innovation,technology


In [43]:
# add url to df
url = xSummaries['article']
scoresdf.insert(0, 'article', url)

In [47]:
scoresdf

,article,summary,oldScore,modBoW,weightScore,topic1,topic2
0,https://datainnovation.org/2022/11/visualizing...,economist created visualization showing politi...,0.50,0.62,0.21,economics,data
1,https://venturebeat.com/ai/stability-ai-looks-...,among organization building generative technol...,0.85,0.58,0.66,technology,innovation
2,https://venturebeat.com/ai/openai-ceo-admits-c...,watch demand session today watch herealtman la...,0.72,0.44,0.29,public,ai
3,https://venturebeat.com/ai/generative-ai-may-b...,generative going completely alter marketing la...,0.92,0.57,0.51,innovation,application
4,https://www.businessinsider.com/moderna-ceo-in...,personalized cancer vaccine could provide new ...,0.71,0.44,0.57,innovation,technology
...,...,...,...,...,...,...,...
120,https://www.cbsnews.com/news/spacex-launch-tod...,spacex falcon rocket carrying gps navigation s...,0.81,0.42,0.55,technology,public
121,https://www.cbsnews.com/video/chatgpt-jobs-art...,rise could chatgpt take job chatbots artificia...,0.93,0.70,0.75,technology,innovation
122,https://www.cbsnews.com/news/ukraine-military-...,jonas ohman founder ceo blue yellow lithuania ...,0.82,0.35,0.44,data,public
123,https://www.cbsnews.com/news/metaverse-making-...,metaverse asset digital art video game charact...,0.36,0.59,-0.22,games,entertainment


In [46]:
jsonfile = join(JSONPath, "articleScoringExperimentTesting.json")
print(jsonfile)

scoresdf.to_json(jsonfile)
complete(state="updates written: ")

/home/deleidos/Notebooks/JSONData/articleScoringExperimentTesting.json

Cell updates written:  @ 2023-02-14 19:44:19
